<a href="https://colab.research.google.com/github/PHoepner/big-sleep/blob/main/Rerunning_Latents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install big-sleep --upgrade

     |████████████████████████████████| 1.4MB 12.8MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 133kB 45.7MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 7.3MB 51.3MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=206f15b6c21f3e30f3797513d40516b25a17f2cf2361d54d1c14d44fdf977dc6
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=d8c17ca31e56c65a0c940c9dc9ecff15ed7e490b6b55c1d41cdd997db17985e5
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built fire ftfy
ERROR: botocore 1.20.33 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
from tqdm.notebook import trange
from IPython.display import Image, display
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam

from torchvision.utils import save_image

import os
import sys
import subprocess
import signal
import string
import re

from datetime import datetime
from pathlib import Path
from tqdm import tqdm, trange

from big_sleep.ema import EMA
from big_sleep.resample import resample
from big_sleep.biggan import BigGAN
from big_sleep.clip import load, tokenize
from big_sleep import Imagine

SEEDTEXT = 'a cat' #@param {type:"string"}
SEEDTEXT2 = 'a charcoal drawing' #@param {type:"string"}
SEEDTEXT3 = 'a painting by picasso' #@param {type:"string"}
SEEDTEXT4 = 'a gorilla' #@param {type:"string"}
SEEDITERATIONS = 125 #@param {type:"number"}
ITERATIONS = 100 #@param {type:"number"}

def track_path(self, epoch, i, pbar=None):
    total_loss = 0
    if (self.isSeed == True):
      print('Creating Seed Image!')
    
    for _ in range(self.gradient_accumulate_every):
        losses = self.model(self.encoded_texts["max"], self.encoded_texts["min"])
        loss = sum(losses) / self.gradient_accumulate_every
        total_loss += loss
        loss.backward()

    self.optimizer.step()
    self.model.model.latents.update()
    self.optimizer.zero_grad()

    if (i > 1):
        with torch.no_grad():
            self.model.model.latents.eval()
            losses = self.model(self.encoded_texts["max"], self.encoded_texts["min"])
            top_score, best = torch.topk(losses[2], k = 1, largest = False)
            image = self.model.model()[best].cpu()
            save_image(image, str(self.filename))
            self.model.model.latents.train()
            save_image(image, str(self.filename))
            lats = self.model.model.latents
            lats.best = best # saving this just in case it might be useful
            torch.save(lats, Path(f'./{self.textpath}.{i}.pth'))
            save_image(image, Path(f'./{self.textpath}.{i}.png'))

    return total_loss

def run_Text(inputText, isSeed, setIterations):

  model = Imagine(
      text = inputText,
      save_every = 100,
      lr = .07,
      iterations = setIterations,
      save_progress = True,
      epochs = 1
  )

  model.isSeed = isSeed

  for epoch in trange(1, desc = 'epochs'):
      for i in trange(setIterations, desc = 'iteration'):
          if (model.isSeed == False and i == 0):
            print('Loading Seed Image!')
            model.model.model.latents = torch.load(Path(f'./'+SEEDTEXT.replace(' ', '_')+'.'+str(SEEDITERATIONS - 1)+'.pth'))
            model.optimizer = Adam(model.model.model.latents.parameters(), model.lr)
          model.train_step(epoch, i)
          
          if i != 0:
            track_path(model, epoch, i, None)
            continue

          filename = inputText.replace(' ', '_')
          image = Image(f'./{filename}.png')
          display(image)

run_Text(SEEDTEXT, True, SEEDITERATIONS)
run_Text(SEEDTEXT2, False, ITERATIONS)
run_Text(SEEDTEXT3, False, ITERATIONS)
run_Text(SEEDTEXT4, False, ITERATIONS)

100%|███████████████████████████████████████| 354M/354M [00:11<00:00, 32.0MiB/s]
iteration:   0%|          | 0/125 [00:00<?, ?it/s]


iteration:   1%|          | 1/125 [00:00<00:47,  2.62it/s]

Creating Seed Image!



iteration:   2%|▏         | 2/125 [00:00<00:48,  2.53it/s]

Creating Seed Image!


...


iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Loading Seed Image!



iteration:  99%|█████████▉| 99/100 [01:21<00:00,  1.22it/s]

image updated at "./a_gorilla.png"



epochs: 100%|██████████| 1/1 [01:22<00:00, 82.82s/it]


In [3]:
!apt install gifsicle
from google.colab import files
from google.colab import output

fileName = SEEDTEXT.replace(" ", "_")
fileName2 = SEEDTEXT2.replace(" ", "_")
fileName3 = SEEDTEXT3.replace(" ", "_")
fileName4 = SEEDTEXT4.replace(" ", "_")
fileName2reversed = SEEDTEXT2.replace(" ", "_") + 'reversed'
fileName3reversed = SEEDTEXT3.replace(" ", "_") + 'reversed'
fileName4reversed = SEEDTEXT4.replace(" ", "_") + 'reversed'

!ffmpeg -start_number 1 -i $fileName\.%d.png $fileName\.gif
!ffmpeg -start_number 1 -i $fileName2\.%d.png $fileName2\.gif
!ffmpeg -start_number 1 -i $fileName3\.%d.png $fileName3\.gif
!ffmpeg -start_number 1 -i $fileName4\.%d.png $fileName4\.gif

!ffmpeg -i $fileName2\.gif -vf reverse $fileName2reversed\.gif
!ffmpeg -i $fileName3\.gif -vf reverse $fileName3reversed\.gif
!ffmpeg -i $fileName4\.gif -vf reverse $fileName4reversed\.gif

!gifsicle $fileName\.gif $fileName2\.gif $fileName2reversed\.gif $fileName3\.gif $fileName3reversed\.gif $fileName4\.gif $fileName4reversed\.gif> combo.gif
files.download("combo.gif")

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  gifsicle
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 134 kB of archives.
After this operation, 385 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 gifsicle amd64 1.91-2 [134 kB]
Fetched 134 kB in 0s (1,786 kB/s)
Selecting previously unselected package gifsicle.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../gifsicle_1.91-2_amd64.deb ...
Unpacking gifsicle (1.91-2) ...
Setting up gifsicle (1.91-2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/includ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>